In [ ]:
from Model_quantized import load_quantized_model, test_quantized_model, verify_model_state

import os

# Load base model 

In [5]:
model_path  = os.path.join(os.getcwd(), "my-llama-3.1-8b-local")

In [6]:
model, tokenizer = load_quantized_model(model_path, use_4bit=True)


=== Loading Model from: /mnt/batch/tasks/shared/LS_root/mounts/clusters/apolloml-apac/code/Users/nontaporn.wonglek/LLM/my-llama-3.1-8b-local ===
=== Setting up Quantization Configuration ===
Quantization configs ready
Loading tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading model with 4-bit quantization...


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:42<00:00, 55.62s/it]



✅ Model loaded successfully!
GPU memory used: 5.31 GB / 15.6 GB
Quantization: 4-bit


Test sample

In [7]:
test_success = test_quantized_model(model, tokenizer)


=== Testing Quantized Model ===
Generating response...

✅ Model test successful!
Input: Hello! Can you tell me what is machine learning?
Response: Hello! Can you tell me what is machine learning? Machine learning is a subset of artificial intelligence (AI) that involves training algorithms to make predictions, classify data, or make decisions based on patterns in data. It's a type of learning where machines can learn from data without being explicitly programmed. There are


Check whether model is quantized properly

In [12]:
verify_model_state(model)


=== Model Verification ===
Model type: <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
Model device: cuda:0
Model dtype: torch.float16
✅ Model is quantized with BitsAndBytes


True

# Prepare data and cross-check quality for fine-tuning

In [13]:
data_path = r"/home/azureuser/cloudfiles/code/Users/nontaporn.wonglek/LLM/data/llama_marketing_training.jsonl"

from data_prep import prepare_your_data_jsonl

In [14]:
train_dataset = prepare_your_data_jsonl(data_path, tokenizer)

=== Step 2: Preparing Training Data (JSONL) ===
✅ Loaded 14 examples from /home/azureuser/cloudfiles/code/Users/nontaporn.wonglek/LLM/data/llama_marketing_training.jsonl
✅ Formatted 14 examples

=== Sample Formatted Text ===
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a marketing expert who creates engaging marketing content.<|eot_id|><|start_header_id|>user<|end_header_id|>

Write marketing content about: How Content Audits Help The...


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 161.89 examples/s]

✅ Tokenized dataset created: 14 samples


# Set up LoRA for fine-tuning quantized model

In [ ]:
from Lora_finetune import setup_lora_pipeline, setup_training, complete_finetuning_pipeline, quick_test

In [16]:
peft_model = setup_lora_pipeline(model)

Setting up LoRA for fine-tuning...
=== Step 3: Preparing Model for LoRA ===
✅ Model prepared for k-bit training
✅ Gradient checkpointing enabled

=== Creating LoRA Configuration ===
✅ LoRA config created
   - Rank: 8
   - Alpha: 16
   - Target modules: 7

=== Applying LoRA to Model ===
trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605

GPU memory after LoRA: 7.36 GB / 15.6 GB

LoRA setup complete
✅ Model is ready for fine-tuning


In [17]:
trainer = setup_training(peft_model, tokenizer, train_dataset)

Setting up training configuration...
=== Step 4: Creating Training Configuration ===
✅ Training arguments configured
   - Epochs: 10
   - Batch size: 1
   - Learning rate: 0.0003
   - Effective batch size: 4

=== Creating Trainer ===
✅ Trainer created successfully

✅ Training setup complete
✅ Current GPU memory: 7.36 GB / 15.6 GB


/mnt/batch/tasks/shared/LS_root/mounts/clusters/apolloml-apac/code/Users/nontaporn.wonglek/LLM/Lora_finetune.py:149: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
success = complete_finetuning_pipeline(trainer, peft_model)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


Starting complete fine-tuning pipeline...
Starting Fine-tuning
Training in progress...
--------------------------------------------------


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
2,2.749500
4,2.998600
6,2.482900
8,2.193900
10,1.845700
12,1.694100
14,1.517200
16,1.421100
18,1.289900
20,1.387600


Fine-tuning completed successfully!
Training time: 5.1 minutes
Final GPU memory: 7.52 GB / 15.6 GB

Saving fine-tuned model...


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


✅ Model saved to: ./marketing_lora_finetuned
✅ LoRA adapters and tokenizer saved

FINE-TUNING COMPLETE
✅ Your marketing content generator is ready!
✅ Saved at: ./marketing_lora_finetuned


In [23]:
quick_test(peft_model, tokenizer)

🔥 Quick Test: Write marketing content about: How AI is Revolutionizing Customer Experience
--------------------------------------------------
🧪 Testing Your Marketing Content Generator

🎯 Test 1: Write marketing content about: How AI is Revolutio...
------------------------------------------------------------
📝 Generated Content:

As a marketer, you know that customer experience is crucial to your business’s success. But delivering exceptional customer experiences is hard. And it’s getting harder. Customers have higher expectations than ever before, and they want to be able to resolve issues on their own. But as a business, you still want to provide them with personalized support that feels human. That’s where AI comes in. By integrating AI into your customer service strategy, you can deliver faster, more personalized su...

